In [1]:
import os
import numpy as np
import torch

from tqdm import tqdm
from torch.utils.data import DataLoader
from rouge_score import rouge_scorer


from model import Model, torch_load_all
from config import CNNConfig
from data_utils import ESDataset, collate_fn

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeLsum'], use_stemmer=True)

def torch_save(dir, save_dict):
    os.makedirs(dir, exist_ok=True)

    for name in save_dict:
        torch.save(save_dict[name], os.path.join(dir, name + '.pt'))
    
def torch_load_all(dir):
    save_dict = {}
    for name in os.listdir(dir):
        save_dict[name.replace('.pt', '')] = torch.load(os.path.join(dir, name), map_location=torch.device('cpu'))

    return save_dict


def get_test_loader(tokenizer, docs, config):
    encodings = []
    for doc in docs:
        encodings.append(tokenizer(doc[:config.MAX_DOC_LEN], truncation=True,
                                   max_length=config.MAX_SEQ_LEN, padding='max_length'))
    
    test_dataset = ESDataset(encodings)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

    return test_loader

def get_all_probs(model, tokenizer, all_doc, config):
    res = []
    test_loader = get_test_loader(tokenizer, all_doc, config=config)
    for item in tqdm(test_loader):
        ids = item['input_ids'].to(config.device)
        document_mask = item['document_mask'].to(config.device)
        attention_mask = item['attention_mask'].to(config.device)
        prob = model(ids, document_mask, attention_mask)[0].tolist()
        res.append(prob)
            
    return res

def calculateSimilarity(sentence, doc):
    score = scorer.score('\n'.join(doc), sentence)
    return np.mean([score['rouge2'][2], score['rougeLsum'][2]])

def choose_summary_mmr(doc, prob, k=3, alpha=0.9):
    prob = np.array(prob)
    idx = [np.argmax(prob)]
    prob[idx[0]] = 0
    summary = [doc[idx[0]]]

    while len(idx) < min(k, len(doc)):
        mmr = -100 * np.ones_like(prob)
        for i, sent in enumerate(doc):
            if prob[i] != 0:
                mmr[i] = alpha * prob[i] - (1-alpha) * calculateSimilarity(sent, summary)
        pos = np.argmax(mmr)
        prob[pos] = 0
        summary.append(doc[pos])
        idx.append(pos)
    summary = sorted(list(zip(idx, summary)), key=lambda x: x[0])
    return [x[1] for x in summary]

def choose_summary(doc, prob, k=3):
    idx = torch.topk(torch.tensor(prob), k=k).indices.tolist()
    return [doc[i] for i in sorted(idx)]

def choose_all_summary(docs, all_probs, k=3):
    summaries = []
    for i, doc in enumerate(docs):
        prob = all_probs[i]
        idx = torch.topk(torch.tensor(prob), k=k).indices.tolist()
        summaries.append([doc[i] for i in sorted(idx)])
    return summaries


2021-10-11 14:15:44.317200: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/lib64
2021-10-11 14:15:44.317249: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# English version

In [7]:
from nltk.tokenize import sent_tokenize as nltk_sent_tokenize

def sent_tokenize(doc):
    return nltk_sent_tokenize(doc)

# load the trained model
final_dict = torch_load_all('save/cnn/best-model')
config = final_dict['config']
config.device = 'cpu'

bert = config.bert_type.from_pretrained(config.bert_name)
tokenizer = config.tokenizer_type.from_pretrained(config.bert_name)
model = Model(bert, config).to(config.device)
model.load_state_dict(final_dict['model_state_dict'])
model.eval()
a = 0

In [5]:
with open('data/cnndm/example.txt') as f:
    doc = f.read()
doc = sent_tokenize(doc)
# doc

In [6]:
docs = [doc]
probs = get_all_probs(model, tokenizer, docs, config)
sumamries = choose_all_summary(docs, probs, k=3)
sumamries

  0%|          | 0/1 [00:00<?, ?it/s]/mnt/disk1/tan_hm/venv/lib/python3.7/site-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 1/1 [00:00<00:00,  2.06it/s]


[['(CNN) — After a long, golden sunset of being installed on fewer and fewer aircraft, the retirement of older aircraft caused by the Covid-19 pandemic means that when air travel resumes, international first class will be very nearly a thing of the past.',
  'Its replacement is a new generation of superbusiness minisuites, more spacious than regular business class, and with a privacy door to create your own space, but without the over-the-top luxury of first class.',
  'The Qsuite is unique to Qatar Airways, but a growing number of airlines offer or plan to offer superbusiness seats, from Delta to China Eastern, JetBlue to British Airways, Shanghai Airlines to Aeroflot, to the very latest, Air China.']]

# Vietnamese version

In [3]:
from underthesea import sent_tokenize as  sent_tokenize_uts, word_tokenize as word_tokenize_uts

def sent_tokenize(doc):
    return sent_tokenize_uts(doc)

def word_tokenize(doc, format='text'):
    return  word_tokenize_uts(doc, format=format)

# load the trained model
final_dict = torch_load_all('save/vietnews/best-model-f1')
config = final_dict['config']
config.device = 'cpu'

bert = config.bert_type.from_pretrained(config.bert_name)
tokenizer = config.tokenizer_type.from_pretrained(config.bert_name)
model = Model(bert, config).to(config.device)
model.load_state_dict(final_dict['model_state_dict'])
model.eval()
a = 0

Some weights of the model checkpoint at Zayt/viRoberta-l6-h384-word-cased were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at Zayt/viRoberta-l6-h384-word-cased and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and i

In [4]:
with open('data/vietnews/example.txt') as f:
    title, summary, doc = f.read().split('\n\n')
doc = sent_tokenize(word_tokenize(doc))
# doc

['Theo trang The_Guardian ( Anh ) , tuyên_bố này được đưa ra vào cuối cuộc hội_đàm trực_tiếp đầu_tiên giữa Mỹ và Taliban kể từ khi Washington rút quân khỏi Afghanistan hồi cuối tháng 8 vừa_qua , chấm_dứt sự hiện_diện quân_sự kéo_dài 20 năm tại quốc_gia Trung Nam_Á .',
 'Phát_ngôn_viên chính_trị của Taliban_Suhail_Shaheen cho biết cuộc đàm_phán giữa họ và Mỹ ở Doha ( Qatar ) đã đạt được kết_quả tốt_đẹp .',
 'Washington đồng_ý viện_trợ nhân_đạo cho Afghanistan nhưng nhấn_mạnh viện_trợ đó không đồng_nghĩa với việc chính_thức công_nhận chính_quyền Taliban ở Afghanistan .',
 'Trong khi đó , phía Mỹ chỉ tiết_lộ hai bên " đã thảo_luận về việc cung_cấp viện_trợ nhân_đạo trực_tiếp cho người dân Afghanistan " .',
 'Người_phát_ngôn Bộ Ngoại_giao Ned_Price gọi các cuộc thảo_luận là " thẳng_thắn và chuyên_nghiệp " , trong đó Mỹ nhắc lại rằng sẽ đánh_giá Taliban dựa trên hành_động của họ thay_vì lời_nói . "',
 'Phái_đoàn Mỹ tập_trung vào các mối quan_tâm về an_ninh , khủng_bố và sơ_tán an_toàn cho c

In [5]:
docs = [doc]
probs = get_all_probs(model, tokenizer, docs, config)
sumamries = choose_all_summary(docs, probs, k=2)
sumamries

  0%|          | 0/1 [00:00<?, ?it/s]/mnt/disk1/tan_hm/venv/lib/python3.7/site-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 1/1 [00:00<00:00,  6.73it/s]


[['Theo trang The_Guardian ( Anh ) , tuyên_bố này được đưa ra vào cuối cuộc hội_đàm trực_tiếp đầu_tiên giữa Mỹ và Taliban kể từ khi Washington rút quân khỏi Afghanistan hồi cuối tháng 8 vừa_qua , chấm_dứt sự hiện_diện quân_sự kéo_dài 20 năm tại quốc_gia Trung Nam_Á .',
  'Washington đồng_ý viện_trợ nhân_đạo cho Afghanistan nhưng nhấn_mạnh viện_trợ đó không đồng_nghĩa với việc chính_thức công_nhận chính_quyền Taliban ở Afghanistan .']]